In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (18975 Collins Ave #4702, Sunny Isles Beach, F...
2     (2900 NE 7th Ave Ph 4701, Miami, FL 33137, USA...
3     (430 Grand Bay Dr APT 1107, Key Biscayne, FL 3...
4     (18911 Collins Ave APT 601, Sunny Isles Beach,...
5     (199 Ocean Ln Dr 200 202, Key Biscayne, FL 331...
6     (733 Crandon Blvd APT 206, Key Biscayne, FL 33...
7     (9601 Collins Ave APT 1102, Bal Harbour, FL 33...
8     (488 NE 18th St #4312, Miami, FL 33132, USA, (...
9     (19925 NE 39th Pl APT 602, Aventura, FL 33180,...
10    (19111 Collins Ave, Sunny Isles Beach, FL 3316...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Residences by Armani Casa\r18975 Collins Ave 4...,5/4/2022,10/11/2022,120,"$4,850,000.00",4062,"$1,527.08",Fabio Lopes,Coldwell Banker Realty,Sergio Waissmann,Fortune International Realty,Residences by Armani Casa 18975 Collins Ave 47...,Residences by Armani Casa,18975 Collins Ave 4702 Sunny Isles Beach,"(18975 Collins Ave #4702, Sunny Isles Beach, F...","(25.9521284, -80.11970810000001, 0.0)",25.952128,-80.119708,0.0
2,Biscayne Beach\r2900 NE 7th Ave 4701\rMiami,5/17/2022,10/12/2022,125,"$3,500,000.00",2691,"$1,300.63",Monica Defortuna,Fortune International Realty,Lourdes Gutierrez,Compass Florida LLC,Biscayne Beach 2900 NE 7th Ave 4701 Miami,Biscayne Beach,2900 NE 7th Ave 4701 Miami,"(2900 NE 7th Ave Ph 4701, Miami, FL 33137, USA...","(25.8048485, -80.18598399999999, 0.0)",25.804848,-80.185984,0.0
3,Grand Bay Tower\r430 Grand Bay Dr 1107\rKey Bi...,7/6/2022,10/12/2022,47,"$3,450,000.00",2840,"$1,214.79",Monica Slodarz,Compass Florida LLC,Monica Slodarz,Compass Florida LLC,Grand Bay Tower 430 Grand Bay Dr 1107 Key Bisc...,Grand Bay Tower,430 Grand Bay Dr 1107 Key Biscayne,"(430 Grand Bay Dr APT 1107, Key Biscayne, FL 3...","(25.6930959, -80.1579994, 0.0)",25.693096,-80.157999,0.0
4,Ocean Three Condo\r18911 Collins Ave 601\rSunn...,12/13/2021,10/12/2022,241,"$3,000,000.00",3945,$760.46,Vanessa Frank PA,One Sotheby's Int'l Realty,Alain Arwas,United Realty Group Inc,Ocean Three Condo 18911 Collins Ave 601 Sunny ...,Ocean Three Condo,18911 Collins Ave 601 Sunny Isles Beach,"(18911 Collins Ave APT 601, Sunny Isles Beach,...","(25.9509344, -80.11988579999999, 0.0)",25.950934,-80.119886,0.0
5,Commodore Club South\r199 Ocean Lane Dr 200-20...,8/1/2022,10/12/2022,43,"$2,500,000.00",2825,$884.96,Victoria Zamorano,Realty One Group Evolution,Marilyn Borroto,BHHS EWM Realty,Commodore Club South 199 Ocean Lane Dr 200-202...,Commodore Club South,199 Ocean Lane Dr 200-202 Key Biscayne,"(199 Ocean Ln Dr 200 202, Key Biscayne, FL 331...","(25.6992732, -80.15724949999999, 0.0)",25.699273,-80.157249,0.0
6,Lake Villa 2\r733 Crandon Blvd 206\rKey Biscayne,3/5/2022,10/11/2022,191,"$2,351,000.00",2470,$951.82,Lucrecia Arana Lindemann,Dezer Platinum Realty LLC,Brigitte Nachtigall,Great Properties Int'l LLC,Lake Villa 2 733 Crandon Blvd 206 Key Biscayne,Lake Villa,2 733 Crandon Blvd 206 Key Biscayne,"(733 Crandon Blvd APT 206, Key Biscayne, FL 33...","(25.6872981, -80.16242369999999, 0.0)",25.687298,-80.162424,0.0
7,Majestic Tower\r9601 Collins Ave 1102\rBal Har...,8/3/2022,10/12/2022,9,"$2,300,000.00",2160,"$1,064.81",Melissa Rubin,Compass Florida LLC,Joelle Oiknine,One Sotheby's Int'l Realty,Majestic Tower 9601 Collins Ave 1102 Bal Harbour,Majestic Tower,9601 Collins Ave 1102 Bal Harbour,"(9601 Collins Ave APT 1102, Bal Harbour, FL 33...","(25.8876563, -80.1222319, 0.0)",25.887656,-80.122232,0.0
8,Aria on the Bay\r488 NE 18th St 4312\rMiami,6/30/2022,10/13/2022,66,"$1,910,000.00",2018,$946.48,Ross Milroy,The Corcoran Group,Alessandro Saponaro,AFS Living,Aria on the Bay 488 NE 18th St 4312 Miami,Aria on the Bay,488 NE 18th St 4312 Miami,"(488 NE 18th St #4312, Miami, FL 33132, USA, (...","(25.793483, -80.1869722, 0.0)",25.793483,-80.186972,0.0
9,Porto Vita\r19925 NE 39th Pl 602\rAventura,3/28/2022,10/11/2022,123,"$1,560,000.00",2850,$547.37,Patricia Genis,One Sotheby's Int'l Realty,Patricia Genis,One Sotheby's Int'l Realty,Porto Vita 19925 NE 39th Pl 602 Aventura,Porto Vita,19925 NE 39th Pl 602 Aventura,"(19925 NE 39th Pl APT 602, Aventura, FL 33180,...","(25.9599315, -80.1255386, 0.0)",25.959931,-80.125539,0.0
10,O0 cean Two Condo\r19111 Collins Ave 3307\rSun...,4/11/2022,10/12/2022,172,"$1,525,000.00",1900,$802.63,Wendy Cohen,One Sotheby's Int'l Realty,Daniela Levtov,NG International Realty LLC,O0 cean Two Condo 19111 Collins Ave 3307 Sunny...,O

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [11]:
df.at[10,'building_name']=('Ocean Two')
df.at[10,'address_only']=('19111 Collins Ave')

In [12]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('October 2nd - October 8th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [15]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [16]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [17]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_100822
